## Task 2

* Implement simple fully-convolutional neural architecture for classification. Make sure it is small enought to run on your home machine.
* Provide dataset visulization.
* Provide train/test split and validation

In [50]:
import time

In [17]:
# download data and split it to batches
import torchvision

tensor = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train = torchvision.datasets.FashionMNIST('data/', download=True, train=True, transform=tensor)
test  = torchvision.datasets.FashionMNIST('data/', download=True, train=False, transform=tensor)

train_loader = torch.utils.data.DataLoader(train,
                                           batch_size=40,
                                           shuffle=True)

test_loader  = torch.utils.data.DataLoader(test,
                                           batch_size=40,
                                           shuffle=True)

In [75]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    """
    FCN
    """
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 20, 5, padding=2)
        self.norm1 = nn.BatchNorm2d(20)
        
        self.conv2 = nn.Conv2d(20, 20, 5)
        self.norm2 = nn.BatchNorm2d(20)
        
        self.conv3 = nn.Conv2d(20, 10, 1)
        self.norm3 = nn.BatchNorm2d(10)
        
        self.lin = nn.Linear(24 * 24 * 10, 10)

    def forward(self, x):
        x = self.norm1(F.relu(self.conv1(x)))
        x = self.norm2(F.relu(self.conv2(x)))
        x = self.norm3(F.relu(self.conv3(x)))
        #x = x.view(x.size())
        x = x.view(40, 24 * 24 * 10)
        
        return F.softmax(self.lin(x))
        

In [76]:
FCN = Model()

In [77]:
loss = torch.nn.MSELoss()#CrossEntropyLoss()
optimizer = torch.optim.SGD(FCN.parameters(), lr=0.1)

In [82]:
%%time

num_epoch = 3
for epoch in range(num_epoch):
    for X, y in train_loader:
        y_est = FCN(X)

        y_vec = torch.zeros(y.shape[0], 10)
        y_vec[torch.arange(y.shape[0]), y] = 1.0
        loss_val = loss(y_est, y_vec)

        optimizer.zero_grad()
        loss_val.backward()

        optimizer.step()
    

CPU times: user 27min 1s, sys: 3min 8s, total: 30min 9s
Wall time: 5min 19s


In [83]:
%%time

score = 0
for X, y in test_loader:
    y_est = FCN(X)
    #print(y_est)
 
    
    classes = torch.argmax(y_est, dim=1)
    
    score += torch.sum(y==classes).item()

print("Accuracy = {}".format(score / len(test)))

Accuracy = 0.898
CPU times: user 29.5 s, sys: 7.99 s, total: 37.4 s
Wall time: 9.37 s
